# QCUT02 : Check quality results in Spectractor

- author Sylvie Dagoret-Campagne
- creation date 2026-01-16 : version vrun2026_v01
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.qualitycuts import scatter_datetime
from mysitcom.auxtel.qualitycuts import strip_datetime
from mysitcom.auxtel.qualitycuts import bar_counts_by_night
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.qualitycuts import stripplot_target_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import  plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import summarize_dccd_chi2

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_QCUT02"
prefix = "qcut02"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from QCUT00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `QCUT00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec[["id","FILTER"]]

## Targets in our data

In [ ]:
List_Of_Targets = df_spec["TARGET"].unique()
NTARGETS = len(List_Of_Targets)
print(NTARGETS,List_Of_Targets)

## Special study on Star Color (Spectral type)
- Load magnitudes from external file
- the magnitudes have been computed in another notebooks in ../2025-10-29-TOOLS



In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df_targets_mag = pd.read_csv(targets_mag_files,index_col=0)      
df_targets_mag = df_targets_mag.sort_values(by="y")

### palette with SED type

In [ ]:
df_col = df_targets_mag.copy()
df_col = df_col.sort_values(by="B_V")

SpT = df_col["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # garde l'ordre d'apparition
N_types = len(unique_types)


# Associe chaque type spectral à un entier
type_to_idx = {t: i for i, t in enumerate(unique_types)}
idx = np.array([type_to_idx[t] for t in SpT])

# Crée la colormap
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# Colorbar horizontale
fig, ax = plt.subplots(figsize=(14, 0.4), layout="constrained")
cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=ax,
    orientation="horizontal",
    ticks=range(N_types),
)
cbar.ax.set_xticklabels(unique_types, rotation=45, ha="right")
cbar.set_label("Spectral Type", fontsize=12,labelpad=10)

plt.show()

In [ ]:
df_targets_mag.head() 

### Target ordered by colors in the input file

In [ ]:
sorted_targets =  list(df_targets_mag.index) 
print(sorted_targets)

### Target ordered by colors in our data  file

In [ ]:
# order pf selected target by magnitude Y
order_selected_targets = [t for t in sorted_targets if t in df_spec["TARGET"].unique()]
print(order_selected_targets)

### build a palette for colors

In [ ]:
# --- Palette personnalisée cohérente avec la colormap jet ---
target_to_color = {}
for target in order_selected_targets:
    sp_type = df_targets_mag.loc[target, "Sp_T"]
    rgba = cmap(norm(type_to_idx[sp_type]))

    # Convert to pure Python floats
    target_to_color[target] = tuple(float(c) for c in rgba)

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="empty",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="empty – all targets (color SED-type)",
    tag = tag,
    target_palette=target_to_color
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-empty_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter_colorsedtype(
    df=df_spec,
    filter_select="empty",
    per_target=False,
    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    target_palette=target_to_color,
    suptitle="empty – histograms",
    tag = tag,
)
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_hosto_all-targets_filter-empty_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="OG550_65mm_1",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="OG550 – all targets (color SED-type)",
    tag = tag,
    target_palette=target_to_color
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-og550_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter_colorsedtype(
    df=df_spec,
    filter_select="OG550_65mm_1",
    per_target=False,

    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    target_palette=target_to_color,
    suptitle="OG550 – histograms",
    tag = tag,
)
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_hosto_all-targets_filter-og550_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="BG40_65mm_1",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="BG40 – all targets (color SED-type)",
    tag = tag,
    target_palette=target_to_color
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-bg40_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter_colorsedtype(
    df=df_spec,
    filter_select="BG40_65mm_1",
    per_target=False,

    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    target_palette=target_to_color,
    suptitle="BG40 – histograms",
    tag = tag,
)
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_hosto_all-targets_filter-bg40_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

### Per target vs time

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="empty",
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="empty – all targets (color SED-type)",
    per_target=True,
    tag = tag,
    target_palette=target_to_color
)

plt.show()

### Understand missing targets

Missing trget that are not in file `targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"`

In [ ]:
ListOfMissingTargets = set(List_Of_Targets) - set(sorted_targets)

In [ ]:
from getCalspec import getCalspec
from getCalspec.getCalspec import getCalspecDataFrame

In [ ]:
for missing_target in ListOfMissingTargets:
    result = getCalspec.is_calspec(missing_target)
    print(f"missing target {missing_target} ==> is_Calspec = {result}")

## Summary

In [ ]:
df_summary = summarize_dccd_chi2(df_spec)
df_formatted = df_summary.copy()
df_formatted["mean_DCCD"]   = df_formatted["mean_DCCD"].map("{:.1f}".format)
df_formatted["sigma_DCCD"]  = df_formatted["sigma_DCCD"].map("{:.1f}".format)
df_formatted["mean_CHI2"]   = df_formatted["mean_CHI2"].map("{:.0f}".format)
df_formatted["sigma_CHI2"]  = df_formatted["sigma_CHI2"].map("{:.0f}".format)

In [ ]:
df_summary["is_calspec"] = df_summary["TARGET"].apply(getCalspec.is_calspec)

In [ ]:
df_summary.head()

### Summary using groupby
(no .reset_index)

In [ ]:
summary = (
    df_spec
    .groupby(["TARGET", "FILTER"])
    .agg(
        mean_DCCD=("D_CCD [mm]", "mean"),
        sigma_DCCD=("D_CCD [mm]", "std"),
        mean_CHI2=("CHI2_FIT", "mean"),
        sigma_CHI2=("CHI2_FIT", "std"),
    )
)

In [ ]:
summary["mean_DCCD"]   = summary["mean_DCCD"].map("{:.1f}".format)
summary["sigma_DCCD"]  = summary["sigma_DCCD"].map("{:.1f}".format)
summary["mean_CHI2"]   = summary["mean_CHI2"].map("{:.0f}".format)
summary["sigma_CHI2"]  = summary["sigma_CHI2"].map("{:.0f}".format)

In [ ]:
summary["is_calspec"] = summary.index.get_level_values("TARGET").map(getCalspec.is_calspec)

In [ ]:
summary

## View Spectractor results in Butler

In [ ]:
target_sel ="HD36780"